In [1]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
import datetime

import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [2]:
%env AWS_SECRET_ACCESS_KEY

'82pgo1H+eurVJr548ZNAuYbqzCewoSojQ4B90TWJ'

In [3]:
kinesis.list_streams()

{'HasMoreStreams': False,
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '41',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 01 Feb 2018 20:24:39 GMT',
   'server': 'Apache-Coyote/1.1',
   'x-amz-id-2': 'pz6qHS6uc/6BEnnsgGNShCUnomkR6OXZpx3ylbBHuLIiB/ucBUPsDmfbJO68PTVGrEMfAK2Q8WHWmUp2vKYdkZAVHoZoFzODa1UbIJj6TJ4=',
   'x-amzn-requestid': 'ceea6709-ff67-48e1-9c07-1ece970d3eb4'},
  'HTTPStatusCode': 200,
  'RequestId': 'ceea6709-ff67-48e1-9c07-1ece970d3eb4',
  'RetryAttempts': 0},
 'StreamNames': []}

### Redis Test

In [4]:
REDIS.get('my_incrementer')

b'42'

In [5]:
#To pass a dictionary:
import numpy as np
import json

model_params = {
    'C': list(np.logspace(-3,3,7)),
    'penalty': 'l1',
    'solver' : 'newton-cg'
}

REDIS.set('model_params', json.dumps(model_params))

True

#### Build Company dataframe

In [20]:
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
stock_tickers

['AAPL', 'FB', 'GOOG']

In [17]:
#Manager Log
# Need to send Number of Companies, Rate
now = datetime.datetime.now()
now

datetime.datetime(2018, 2, 1, 20, 29, 26, 66148)

### IEX Code -------------------------------------------------------

In [ ]:
#Imports
import redis
import urllib.request
import json
import numpy as np
import pandas as pd
import schedule
import time
import boto3
import datetime

import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table

kinesis = boto3.client('kinesis', region_name='us-east-1')


#Connect to Redis-DataStore
REDIS = redis.Redis(host='data_store')

In [3]:
User = %env DB_USER
password = %env DB_PWD
dbname = %env DB_NAME

In [4]:
#Want to create a simple list, and then convert the time stamps
attributes = ['latestUpdate',
             'companyName',
             'latestPrice',
              'latestVolume',
              'marketCap',
              'open',
              'previousClose',
              'sector',
              'high',
              'low',
              'ytdChange',
              'peRatio',
              'week52High',
              'week52Low'
               ]
def filter_stock_attributes(data):
    output = {}
    #Choose filter attributes
    for element in attributes:
        output[element]=data[element]
    
    #Convert time
    ctime = output['latestUpdate']/1000
    new_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ctime))
    output['latestUpdate']= new_time
    
    return output

#Create function to fetch the stock data. This is to prepare for the schedule.
def fetch_stock_data(stocks=[]):
    
    for ticker in stocks:
        url = "https://api.iextrading.com/1.0/stock/{}/quote".format(ticker)
        response = urllib.request.urlopen(url)
        str_response = response.read().decode('utf-8')
        obj = json.loads(str_response)
        filtered = filter_stock_attributes(obj)
        #Add counter to count stocks. 
        REDIS.incr('IEX_Stock_Count')
        #<----- Insert to Kinesis Stream ------->
        #response = kinesis.put_record(StreamName="IEX_Stream", Data=json.dumps(filtered), PartitionKey="partitionkey")
        print('---------------------------------')
        #print(response)
        print(filtered)

#Setup Log

global past_stock_count
past_stock_count = 0

#Setup Database
engine = create_engine('mysql+mysqlconnector://{}:{}@db:3306/{}'.format(User, password, dbname), echo=False)
conn = engine.connect()

#Create table object
meta = MetaData(engine,reflect=True)
table = meta.tables['log']

def log():
    #Need to log: Time, Source, Current Count, Count Diff
    now = datetime.datetime.now()
    current_stock_count = int(REDIS.get('IEX_Stock_Count'))
    
     
    global past_stock_count
    stock_count_diff = current_stock_count - past_stock_count
    past_stock_count = current_stock_count
    
    #Need to log these items to a database.
    ins = table.insert().values(
            log_time = now,
            Source = 'IEX',
            Current_Count=current_stock_count,
            Count_Diff=stock_count_diff
               )
    conn.execute(ins)
    print('Logged Data')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:57: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.


In [5]:
#Read Data-Store
companies = json.loads(REDIS.get('companies').decode())
stock_tickers = list(companies.keys())
REDIS.set('IEX_Stock_Count', 0)

#Setup Schedule
schedule.clear()
schedule.every(10).seconds.do(fetch_stock_data, stocks=stock_tickers)
schedule.every(60).seconds.do(log)

#Execute
while True:
    schedule.run_pending()
    time.sleep(1)

---------------------------------
{'latestUpdate': '2018-02-02 20:59:07', 'companyName': 'Apple Inc.', 'latestPrice': 160.305, 'latestVolume': 80027976, 'marketCap': 815480512080, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.8, 'ytdChange': -0.026007198420991466, 'peRatio': 17.42, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:07', 'companyName': 'Facebook Inc.', 'latestPrice': 190.12, 'latestVolume': 23861360, 'marketCap': 552452359170, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194.21, 'low': 190.22, 'ytdChange': 0.06432587366332276, 'peRatio': 30.91, 'week52High': 195.32, 'week52Low': 130.3}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:02', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.78, 'latestVolume': 4475051, 'marketCap': 773161893079, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 112

---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4759225, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123.07, 'low': 1107.278, 'ytdChange': 0.09643192488262915, 'peRatio': None, 'week52High': 1186.89, 'week52Low': 792}
---------------------------------
{'latestUpdate': '2018-02-02 21:00:08', 'companyName': 'Apple Inc.', 'latestPrice': 160.4, 'latestVolume': 85618081, 'marketCap': 815963782400, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.55, 'ytdChange': -0.026007198420991466, 'peRatio': 17.43, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25522873, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194

---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4759225, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123.07, 'low': 1107.278, 'ytdChange': 0.09643192488262915, 'peRatio': None, 'week52High': 1186.89, 'week52Low': 792}
Logged Data
---------------------------------
{'latestUpdate': '2018-02-02 21:00:08', 'companyName': 'Apple Inc.', 'latestPrice': 160.4, 'latestVolume': 85639263, 'marketCap': 815963782400, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.428, 'ytdChange': -0.026007198420991466, 'peRatio': 17.43, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25522873, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology'

---------------------------------
{'latestUpdate': '2018-02-02 21:00:08', 'companyName': 'Apple Inc.', 'latestPrice': 160.4, 'latestVolume': 85681658, 'marketCap': 815963782400, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.33, 'ytdChange': -0.026007198420991466, 'peRatio': 17.43, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25550570, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194.21, 'low': 190.22, 'ytdChange': 0.06432587366332276, 'peRatio': 30.92, 'week52High': 195.32, 'week52Low': 130.3}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4759225, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123

---------------------------------
{'latestUpdate': '2018-02-02 21:00:08', 'companyName': 'Apple Inc.', 'latestPrice': 160.4, 'latestVolume': 85724096, 'marketCap': 815963782400, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.33, 'ytdChange': -0.026007198420991466, 'peRatio': 17.43, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25550570, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194.21, 'low': 190.21, 'ytdChange': 0.06432587366332276, 'peRatio': 30.92, 'week52High': 195.32, 'week52Low': 130.3}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4759225, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123

---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4759225, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123.07, 'low': 1107.278, 'ytdChange': 0.09643192488262915, 'peRatio': None, 'week52High': 1186.89, 'week52Low': 792}
---------------------------------
{'latestUpdate': '2018-02-02 21:00:08', 'companyName': 'Apple Inc.', 'latestPrice': 160.4, 'latestVolume': 85745331, 'marketCap': 815963782400, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.33, 'ytdChange': -0.026007198420991466, 'peRatio': 17.43, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25564441, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194

---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25564441, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194.21, 'low': 190.13, 'ytdChange': 0.06432587366332276, 'peRatio': 30.92, 'week52High': 195.32, 'week52Low': 130.3}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4760489, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123.07, 'low': 1107.278, 'ytdChange': 0.09643192488262915, 'peRatio': None, 'week52High': 1186.89, 'week52Low': 792}
---------------------------------
{'latestUpdate': '2018-02-02 21:07:05', 'companyName': 'Apple Inc.', 'latestPrice': 160.77, 'latestVolume': 85897246, 'marketCap': 817845993120, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166

---------------------------------
{'latestUpdate': '2018-02-02 21:07:05', 'companyName': 'Apple Inc.', 'latestPrice': 160.77, 'latestVolume': 85961237, 'marketCap': 817845993120, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.33, 'ytdChange': -0.026007198420991466, 'peRatio': 17.48, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25578327, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 194.21, 'low': 190.13, 'ytdChange': 0.06432587366332276, 'peRatio': 30.92, 'week52High': 195.32, 'week52Low': 130.3}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4760489, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 112

---------------------------------
{'latestUpdate': '2018-02-02 20:59:59', 'companyName': 'Alphabet Inc.', 'latestPrice': 1112.53, 'latestVolume': 4760489, 'marketCap': 772988192551, 'open': 1121.95, 'previousClose': 1167.7, 'sector': 'Technology', 'high': 1123.07, 'low': 1107.278, 'ytdChange': 0.09643192488262915, 'peRatio': None, 'week52High': 1186.89, 'week52Low': 792}
---------------------------------
{'latestUpdate': '2018-02-02 21:07:05', 'companyName': 'Apple Inc.', 'latestPrice': 160.77, 'latestVolume': 86025323, 'marketCap': 817845993120, 'open': 166.34, 'previousClose': 167.78, 'sector': 'Technology', 'high': 166.8, 'low': 160.24, 'ytdChange': -0.026007198420991466, 'peRatio': 17.48, 'week52High': 180.1, 'week52Low': 127.78}
---------------------------------
{'latestUpdate': '2018-02-02 20:59:56', 'companyName': 'Facebook Inc.', 'latestPrice': 190.17, 'latestVolume': 25578327, 'marketCap': 552597649607, 'open': 192.1, 'previousClose': 193.09, 'sector': 'Technology', 'high': 19

KeyboardInterrupt: 